In [ ]:
from main import *
from models import *
from tqdm import tqdm

beta_1 = 1e-4
beta_T = 0.02
T = 1000
shape = (44, 126, 16)
device = torch.device('cuda')

model = DiffusionModel(device, beta_1, beta_T, T)
process = DiffusionProcess(beta_1, beta_T, T, model, device, shape)
optim = torch.optim.Adam(model.parameters(), lr = 1e-4)

total_iteration = 500
current_iteration = 0

train_sub = [1]

train_x0, train_y0 = make_train_dataset(train_sub, 0)
train_stft = Stft_dataset(train_x0, train_y0)

dataloader = torch.utils.data.DataLoader(train_stft, batch_size = 16, drop_last = True, num_workers = 0)

pbar = tqdm(range(total_iteration))


for epoch in pbar:
    losses = []
    num_items = 0
    
    for x, y in dataloader:
        data = x.to(device = device)
        loss = loss_fn(model, x)

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        losses.append(loss.item())
        num_items += x.shape[0]
    pbar.set_description("Average loss : {}".format(sum(losses) / num_items))

In [ ]:
sampling_number = 4
only_final = True
process = DiffusionProcess(beta_1, beta_T, T, model, device, shape)
sample = process.sampling(sampling_number, only_final)

In [ ]:
from main import *
import matplotlib.pyplot as plt

sample0 = sample[0]
for stft in sample0:
    plt.plot(stft.cpu())
    plt.show()


generated_signal = return_to_signal(sample)
for signal in generated_signal[0]:
    plt.plot(signal)
    plt.show()

In [ ]:
from main import *

train_sub = [1]
val_sub = [9]
test_sub = [9]

device = 'cuda'
main(train_sub, val_sub, test_sub, device)


In [ ]:
# 실험결과 
# 그냥 stft 바로 집어넣었을 때 loss 0.002 부근

# stft / 10 했을 때 loss 더 줄어든다
